In [1]:
import pickle

import matplotlib.pyplot as plt
import promptlib
import seaborn as sns

from face_recognition.api import get_known_people_from_encodings, get_known_people_from_dataset
from face_recognition.web_prediction import predict
from test.video_prediction import produce_video

method = input('Test by:\n (1) Webcam\n(2) video')
output_faces = []
person_name = ''

if method != '1' and method != '2':
    exit(1)
elif method == '1':
    name = input("Enter your name: ")
    faces = predict()
    names_in_db = get_known_people_from_dataset()
    x = {}
    print(faces)
    for nume in names_in_db:
        x[nume] = (faces.count(nume))
    x['unknown'] = faces.count('unknown')
    print(x)
    keys = [j for j in x.keys()]
    values = [j for j in x.values()]

    sns.set_style('dark')
    acc = x[name] * 100 / sum(values)

    sns.barplot(x=values, y=keys).set_title(f'{name.capitalize()} - acc: {acc}%')
    plt.show()

    print(x)
    exit(5)
elif method == '2':
    prompter = promptlib.Files()
    abs_path_to_video = prompter.file()
    should_be = input("Please enter person name:")
    file_name = abs_path_to_video.split('.')[0].split('/')[-1]
    output_faces = produce_video(abs_path_to_video, should_be)
    prompter.dst()

    print('Done.')


In [2]:
import os


def check_if_safe(should_be, face_list):
    truth_list = []
    if should_be.lower() == 'unknown':
        return False
    for face in face_list:
        if face == should_be:
            truth_list.append(True)
        else:
            truth_list.append(False)

    truth_list = truth_list.sort()
    if truth_list:
        if len(truth_list) < 30:
            return True
        else:
            if not truth_list[30]:
                return False
    return True


def return_results(output_faces, should_be):
    for i in range(len(output_faces)):
        output_faces[i] = output_faces[i].lower()

    safe = check_if_safe(should_be, output_faces)

    if len(output_faces) == 0:
        print('No faces found')
        output_faces = []

    acc = output_faces.count(should_be) * 100 / len(output_faces)
    print(acc)

    known_encodings = get_known_people_from_encodings()
    for i in range(len(known_encodings)):
        known_encodings[i] = known_encodings[i].lower()

    names = {}
    for name in known_encodings:
        names[name] = output_faces.count(name.lower())
    names['unknown'] = output_faces.count('unknown')
    return [file_name.lower(), names, acc, safe]


list = []
if len(output_faces) == 0:
    exit(5)
else:
    list = return_results(output_faces, should_be)
print(list)


if not os.path.isdir(f'results/{should_be}'):
    try:
        os.mkdir(f'results/{should_be}')
    except OSError:
        print("Creation of the directory %s failed" % should_be)

with open(f'results/{should_be}/{list[0]}.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(list, filehandle)

[]


NameError: name 'should_be' is not defined

In [ ]:

import pickle
import matplotlib.pyplot as plt
import os
import seaborn as sns


def plot_all_data():
    directory = 'results'
    objs = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and filename.endswith('data'):
            with open(f, 'rb') as filehandle:
                # read the data as binary data stream
                while 1:
                    try:
                        objs.append(pickle.load(filehandle))
                    except EOFError:
                        break
    sns.set_style('darkgrid')
    for results in objs:
        actual_name = results[0]
        output_faces = results[1]
        safe = results[3]
        print('Safe:', safe)
        if output_faces == 'noface':
            sns.barplot(x=values, y=keys).set_title(actual_name + '- no face found')
            plt.show()
        else:
            acc = results[2]
            print('Accuracy= ', acc, '%')

            keys = [j for j in output_faces.keys()]
            values = [j for j in output_faces.values()]
            if actual_name == 'yt3(2)':
                sns.barplot(x=values, y=keys).set_title(actual_name + 'interchanged train with test')
            else:
                sns.barplot(x=values, y=keys).set_title(actual_name)

            plt.show()


plot_all_data()